<a href="https://colab.research.google.com/github/muhammedshelleh/CAP-5610/blob/HW1/CAP_5610_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#packages for data pre-processing, analysis, and visualization
#NumPy allows users to perform many different types of dataframe analysis
import sklearn
import numpy as np
#Pandas, built on top of NumPy, allows users to pre-process and clean data, along with analysis tools
import pandas as pd
#seaborn allows users to perform multiple visualization techniques
import seaborn as sns
%matplotlib inline 
sns.set_theme(style="darkgrid")
#pyplot, part of matplotlib, allows users to plot various types of data
import matplotlib.pyplot as plt
import math
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

# Task 1 - Q1

In [ ]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape


Before (891, 12) (418, 11) (891, 12) (418, 11)


('After', (891, 10), (418, 9), (891, 10), (418, 9))

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

pd.crosstab(train_df['Title'], train_df['Sex'])
#place outlier titles into separate category, otherwise known as miscellaneous
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Miscellaneous')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()
#change titles to ordinal values
ordinal_titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Miscellaneous": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(ordinal_titles)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,C,3
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,7.9250,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,53.1000,S,3
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,8.0500,S,1


In [ ]:
#drop name and passenger ID
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((891, 9), (418, 9))

In [ ]:
for dataset in combine:
    dataset['TotalFam'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['Singular'] = 0
    dataset.loc[dataset['TotalFam'] == 1, 'Singular'] = 1
    dataset['Class U Age'] = dataset.Age * dataset.Pclass

train_df[['Singular', 'Survived']].groupby(['Singular'], as_index=False).mean()

,Singular,Survived
0,0,0.505650
1,1,0.303538


In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'TotalFam'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'TotalFam'], axis=1)
combine = [train_df, test_df]

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,Singular,Class U Age
0,0,3,0,22.0,7.2500,S,1,0,66.0
1,1,1,1,38.0,71.2833,C,3,0,38.0
2,1,3,1,26.0,7.9250,S,2,1,78.0
3,1,1,1,35.0,53.1000,S,3,0,35.0
4,0,3,0,35.0,8.0500,S,1,1,105.0


In [73]:
ports = train_df.Embarked.dropna().mode()[0]
ports
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(ports)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
1,1,0.553571
2,2,0.389610
0,0,0.339009


In [74]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
train_df['FareRanges'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareRanges', 'Survived']].groupby(['FareRanges'], as_index=False).mean().sort_values(by='FareRanges', ascending=True)

,FareRanges,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [76]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareRanges'], axis=1)
combine = [train_df, test_df]

KeyError: ignored

# Task 1 - Q2

In [80]:
train_df.head(1)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,Singular,Class U Age
0,0,3,0,22.0,0,0,1,0,66.0


In [81]:
test_df.head(1)

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,Singular,Class U Age
0,892,3,0,34.5,0,2,1,1,103.5


Important Features are PassengerID, Pclass, Sex, Age, Fare, Embarked, Title, Singular (vs married/has a family), and Class U Age (combines p-class and age). All of these are significant since they have been cleaned and each provide important details regarding the passengers.

# Task 1 - Q3

In [83]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

#Task 2

In [91]:
x = 5 + 5 + 6 + 2 + 6 + 5
y = 29 + 14+7+10+8+17+15
z = x/y
z #z is the answer for Q1

0.29



1.   For A = 0, + = 30, - = 28, therefore +
2.   For B = 1, + = 10, - = 16, therefore -
3.   For C = 1, + = 15, - = 5, therefore +
4.   For D = 1, + = 6, - = 7, therefore -
5.   For E = 0, + = 2, - = 10, therefore -

Overall, tree will result in class "-"



# Task 3

In [ ]:
# Create Sample 
data={'A':['T','T','T','T','T','F','F','F', 'T', 'T'],'B':['F','T','T','F','T','F','F','F', 'T', 'F'],'ClassLabel':['+','+','+','-','+','-','-','-', '-', '-']}
df=pd.DataFrame (data)
#sort it by ClassLabel
df.sort_values(['ClassLabel'],inplace=True)
print(df)

   A  B ClassLabel
0  T  F          +
1  T  T          +
2  T  T          +
4  T  T          +
3  T  F          -
5  F  F          -
6  F  F          -
7  F  F          -
8  T  T          -
9  T  F          -


In [ ]:
print(df[df.A=='T'],"\n")
print(df[df.A=='F'],"\n")

   A  B ClassLabel
0  T  F          +
1  T  T          +
2  T  T          +
4  T  T          +
3  T  F          -
8  T  T          -
9  T  F          - 

   A  B ClassLabel
5  F  F          -
6  F  F          -
7  F  F          - 



In [ ]:
from math import log2
def entropy(class0, class1):
	return -(class0 * log2(class0) + class1 * log2(class1))

In [ ]:
T = 11 / 20
F = 9 / 20
total_entropy = entropy(T, F)
print('Total Entropy: %.3f bits' % s_entropy)

Total Entropy: 0.993 bits


In [ ]:
A_T = 7 / 10
A_F = 3 / 10
# calculate the entropy of the first group
A_entropy = entropy(A_T, A_F)
print('Column A Entropy: %.3f bits' % A_entropy)

Column A Entropy: 0.881 bits


In [ ]:
B_T = 4 / 10
B_F = 6 / 10
# calculate the entropy of the first group
B_entropy = entropy(B_T, B_F)
print('Column B Entropy: %.3f bits' % B_entropy)

Column B Entropy: 0.971 bits


In [ ]:
gain = total_entropy - (10/20 * A_entropy + 10/20 * B_entropy)
print('Information Gain: %.3f bits' % gain)

Information Gain: 0.067 bits


Decision Tree Would Absolutely Choose Column B, or 'T'

#Task 4